# Project 1 - Sliding Problem
## <font color='red'>Due Date: </font>June 16, 11:59pm

---

* Code is modified from Prof. Nils Napp (Sliding Probelm for AI-Class)

## 1. Defining 3x3 sliding problem environment
No modification should be done in this part.

In [0]:
''' 
Nils Napp
Sliding Probelm for AI-Class
'''

import random
import time



class state:
    """ State of sliding number puzzle
        Contains array of values called 'board' to indicate
        tile positions, and the position of tile '0', which
        indicates the empty space on the board.         """
    
    boardSize=3

    def __init__(self,s=None):

        if s == None:
            
            tiles=range(self.boardSize*self.boardSize).__iter__()
            self.board=[[next(tiles) for i in range(self.boardSize)] for j in range(self.boardSize)]

            #keep track of empty position
            self.position=[0,0]
        
        else:
            #copy the board
            self.board=[]
            for row in s.board:
                self.board.append(list(row))

            #copy the positions    
            self.position=list(s.position)
            
        
    def __str__(self):
        #won't work for larger boards 
        rstr=''
        for row in self.board:
            rstr += str(row) + '\n'
        return rstr
    
    #overload to allow comparison of lists and states with ==
    def __eq__(self,other):
        if isinstance(other, state):
            return self.board == other.board
        elif isinstance(other,list):
            return self.board == other
        else:
            return NotImplemented
 
    def __lt__(self,other):
        if isinstance(other, state):
            return self.h < other.h
        else:
            return NotImplemented
            
    def __cmp__(self,other):
        if isinstance(other, state):
            return cmp(self.h,other.h)
        else:
            return NotImplemented

    #turn into immutable ojbect for set lookup
    def toTuple(self):
        tpl=()
        for row in self.board:
            tpl += (tuple(row),)
        return tpl
    
    #create board from a list or tuple 
    def setBoard(self,brd):
        self.board=brd
        for row in range(self.boardSize):
            for col in range(self.boardSize):
                if self.board[row][col]==0:
                    self.position=[row,col]
                    return None
        raise StandardError('Set board configuration does not have an empy spot!')

## 2. Defining nodes for search graph
No modification should be done in this part.

In [0]:
class node:
    
    nodeCount=0
    
    def __init__(self, p, a, c, s):
        
        #keep track of how many nodes were created
        self.__class__.nodeCount += 1    
        self.nodeID=self.nodeCount
        
        self.parent=p
        self.cost=c
        self.action=a
        self.state=s
        
    #test equivalence Should be state

    def __str__(self):
        rstr= 'NodeID: ' + str(self.nodeID) + '\n'
        if self.parent != None:
            rstr+='Parent: ' + str(self.parent.nodeID) + '\n'
        if self.action != None:
            rstr+='Action: ' + self.action  + '\n'
        rstr+='Cost:   ' + str(self.cost) + '\n'
        rstr+='State:\n' + str(self.state)
        return rstr

    def __cmp__(self,other):
        if isinstance(other,node):
            return cmp(self.h,other.h)
        else:
            return NotImplemented 

            
    def __lt__(self,other):
        if isinstance(other,node):
            return self.h < other.h
        else:
            return NotImplemented 
            
            
def childNode(n, action, problem):
    return node(n,action, n.cost + 1, problem.apply(action,state(n.state)))


## 3. Defining problem
No modification should be done in this part.

Possible actions of our agents are 
* 'U' - Up
* 'L' - Left
* 'D' - Down
* 'R' - Right




In [0]:
#problem
class problem:
    """Class that defines a search problem"""

    def __init__(self):
        self.actions=['U','L','D','R']
        self.initialState=0
        self.goalState=0

    def apply(self,a,s):

        #positions after move, still refers to s.position object
        post=s.position

        #make a copy
        pre=list(post)
        
        #compute post position
        if a == 'U':
            post[0]=max(pre[0]-1,0)
        elif a == 'L':            
            post[1]=max(pre[1]-1,0)
        elif a == 'D':
            post[0]=min(pre[0]+1,s.boardSize-1)
        elif a == 'R':
            post[1]=min(pre[1]+1,s.boardSize-1)
        else:
            print('Undefined action: ' + str(a))
            raise StandardError('Action not defined for this problem!')

        #store the old tile
        tile=s.board[pre[0]][pre[1]]
        
        s.board[pre[0]][pre[1]]=s.board[post[0]][post[1]]
        s.board[post[0]][post[1]]=tile      

 #       print pre, ' ', post,' ',s.board[pre[0]][pre[1]] , '<--', s.board[post[0]][post[1]]      

        return s
        
    def applicable(self,s):
        actionList=[]

        #check if actions are applicable
        #Not in top row
        if s.position[0]>0:
            actionList.append('U')

        #not in left most col
        if s.position[1]>0:
            actionList.append('L')

        #not in bottom row
        if s.position[0]<(s.boardSize-1):
            actionList.append('D')

        #not in right col
        if s.position[1]<(s.boardSize-1):
            actionList.append('R')

        return actionList

    def goalTest(self,s):
        return self.goalState==s    

def applyRndMoves(numMoves,s,p):
    for i in range(numMoves):
        p.apply(p.actions[random.randint(0,3)],s)
    
def solution(node):
    ''' Returns actionList, cost of the solution generated from the node'''

    actions=[]
    cost=node.cost

    while node.parent != None:
        actions.insert(0,node.action)
        node=node.parent    

    return actions, cost        

# Solving Sliding problem
1.   Breadth First Search (BFS) Approach
> *It will find the closest solution, by searching layer-by-layer.*

> The BFS explores an entire layer before progressing.  Each layer consists of a list of nodes.  These lists are looped through, and expanded into a new list based on the available directions to explore for that node at it's given state.  Once a list is exhausted, it is replaced by the new list, and is then expanded.  The cycle ends once 15 layers, or 15 lists have been explored, or when a solution is found.  

2.   Depth First Search (DFS) Approach
> *It will find the first solution, by searching as deep as possible first.*

> The DFS method explores as deeply as possible, before branching out to adjacent nodes.  This was accomplished using recurrence and a for loop.  The for loop would loop through all directions in a function, and call on the function to explore downwards in each one.  Once the function reached it's maximum depth of 15 without finding a solution, it would return "None," allowing the previously halted function above it to explore a new, adjacent path, or return "None" itself to allow the function above it to explore a new path, etc.  


In [0]:
class Searches:    
    
     
    def BFS(self, problem):
      # Recall: Expand a potential board as slowly and thoroughly
      # as possible
      
      # Initialization
      parNode = node(None, None, 0, problem.initialState) # Parent node
      currLayer = [parNode]  # All nodes in our current layer
      nextLayer = []         # All nodes in our next layer
      
      # Run through all options until we find a solution
      # Up to a maximum depth of 15 nodes
      for depth in range(1, 15):
      
        # Exhaust the current layer of options
        #   expNode: Expanding Node
        for expNode in currLayer:

          # Explore all options for this node
          #   currNode: Current Node being explored
          #   dir:      A specific action selected from the exhaustive
          #             available action list
          for dir in problem.applicable(expNode.state):
            currNode = childNode(expNode, dir, problem)
            
            # Check if this node is a solution
            if problem.goalTest(currNode.state):
              return currNode
            
            # Add to the nextLayer list for later expansion
            nextLayer.append(currNode)
        
        # Configure to expand next layer
        currLayer = nextLayer
        nextLayer = []
           
    def recurrDFS(self, node, depth, problem):
      # Call function on ALL possible directions, using recursion...
      # Recall: Since this isn't threaded, it will prioritize depth, fully 
      #         exploring a branch, only breaking when return a solution

      # In case we dig too deep without finding a solution - inform the function
      if depth == 15:
        return None

      for dir in problem.applicable(node.state):
        newNode = childNode(node, dir, problem)

        # We found a solution!
        if problem.goalTest(newNode.state):
          return newNode

        # We need to keep looking...
        else:
          # Use a variable to avoid excess calculations (storage is cheaper)
          nextNode = self.recurrDFS(newNode, depth + 1, problem)
          
          # Checks if the path isn't a dead end.
          if nextNode != None:
            return nextNode
  
    def DFS(self, problem):
      # Solution: Use recurrence to test a whole branch entirely
      originNode = node(None, None, 0, problem.initialState)
      return self.recurrDFS(originNode, 0, problem)


# Test your solution


In [0]:
if __name__ == '__main__':
       
    search = Searches()
    
    p = problem()
    s = state()
    
    p.goalState = state(s)
    
    p.apply('R',s)
    p.apply('R',s)
    p.apply('D',s)
    p.apply('D',s)
    p.apply('L',s)
    
    p.initialState=state(s)
    
    print("Initial State \n", p.initialState)
    
    
### Uncomment for testing BFS solution

#     print('=== BFS  ===')
#     startTime=time.clock()
#     res=search.BFS(p)
#     print(res)
#     print("Time " + str(time.clock()-startTime))
#     print("Explored Nodes: "+ str(node.nodeCount))
 
    
    
#     print("Generating Random Position")
#     si=state(s)
#     applyRndMoves(15,si,p)
#     p.initialState=si
#     print(si)
    
#     startTime=time.clock()
    
#     print('=== BFS  ===')
#     startTime=time.clock()
#     res=search.BFS(p)
#     print(res)
#     print("Time " + str(time.clock()-startTime))
#     print("Explored Nodes: "+ str(node.nodeCount))



### Uncomment for testing DFS solution

    
#    print(p.initialState)
    
    
#     print('=== DFS  ===')
#     startTime=time.clock()
#     res=search.DFS(p)
#     print(res)
#     print("Time " + str(time.clock()-startTime))
#     print("Explored Nodes: "+ str(node.nodeCount))
 
    
    
#     print("Generating Random Position")
#     si = state(s)
#     applyRndMoves(15,si,p)
#     p.initialState=si
#     print(si)
    
#     startTime = time.clock()
    
#     print('=== DFS  ===')
#     startTime=time.clock()
#     res=search.DFS(p)
#     print(res)
#     print("Time " + str(time.clock()-startTime))
#     print("Explored Nodes: "+ str(node.nodeCount))
    

Initial State 
 [1, 2, 5]
[3, 4, 8]
[6, 0, 7]

=== DFS  ===
NodeID: 3743
Parent: 3742
Action: L
Cost:   15
State:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]

Time 0.029075999999996327
Explored Nodes: 3743
Generating Random Position
[1, 5, 8]
[3, 2, 0]
[6, 4, 7]

=== DFS  ===
NodeID: 7486
Parent: 7485
Action: L
Cost:   15
State:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]

Time 0.02707900000000052
Explored Nodes: 7486


# Results

1. **Solution Quality**

> **BFS**, unsurprisingly, found higher quality solutions requiring significantly fewer nodes (*typically 5-8*) to solve the sliding puzzle.  **DFS** generally approached it's maximum (*~14 to 15 nodes*.)

2. **Empirical Runtime Comparison**

> **BFS** performed consistently better (*\~0.002 seconds*) compared to **DFS** (*\~0.02 to 10 seconds*.)  The major difference between the two was the number of nodes explored - DFS typically explored *many* more nodes searching for a solution, and the number required varied *significantly* from run-to-run.

> **Note**: Some exceptions occured, where trivial solutions enabled **DFS** to achieve solution times of *0.0003 seconds* - but this solution required exploring two nodes to reach.  The solve times had a high variance.

# Conclusions
The lack of potential progression with increased inputs put **DFS** at a disadvantage compared to **BFS**.  The sliding problem can, after 15 actions, can *still be just as close to being solved as it was initially*.  

As a result, rushing to a dead-end solution with exponentially more nodes to explore at high depths due to the rapidly branching nature of this problem (*~2 to 3 nodes generated per node*.)  Further, adjacent nodes had a low-likelihood of being close enough to the solution to solve within their significantly limited depth of nodes to explore.  This made **DFS** *incredibly inefficient* compared to **BFS**, which had fewer nodes to explore, and a relatively stable solve depth. 